In [1]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
import json
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time as tm
from dateutil.relativedelta import relativedelta
from datetime import date
import os
import shutil
import re
from selenium.webdriver.common.action_chains import ActionChains


In [130]:
serv_obj = Service("C:\driver\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(service = serv_obj)

# 1. Find all the players name and their page link

## Team page link

In [131]:
pageLink = "https://www.espncricinfo.com/cricketers/team/south-africa-3/alpha-"
lis = []

In [132]:
def makeStandardName(name):
    new_name = " ".join((re.sub("([^a-zA-Z]+|json|png|nbsp|jpg)"," ",name).strip().split())).title()
    return new_name

In [133]:
SCROLL_PAUSE_TIME = 2
ele = None
name_lis = []
page_link = []

for i in range(97,123):
    newPageLink = pageLink + chr(i)
    driver.get(newPageLink)
    driver.maximize_window()
    time.sleep(5)
    
    ele = driver.find_elements(By.CSS_SELECTOR,'span.ds-text-tight-s.ds-font-regular')
    for element in ele:
        if element.get_attribute('innerHTML') == "ALL":
            element.click()
            break
    
    time.sleep(5)
    
    try:
        element = driver.find_element(By.CLASS_NAME,"icon-keyboard_arrow_up-filled")

        actions = ActionChains(driver)
        
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            actions.move_to_element(element).perform()
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height

        obj = driver.find_elements(By.CSS_SELECTOR,".ds-flex.ds-p-4 ")
        lis.append(list(obj))
        
        for ele in list(obj):
            name_lis.append(ele.find_element(By.CLASS_NAME,"ds-text-tight-l").get_attribute('innerHTML'))
            page_link.append(ele.find_element(By.CSS_SELECTOR,'a.ds-leading-none').get_attribute('href'))
        
    except:
        pass

In [134]:
for i in range(len(name_lis)):
    name_lis[i] = makeStandardName(name_lis[i])

In [135]:
df = pd.DataFrame({'Player Name':name_lis,'Page Link':page_link})


In [136]:
dfPlayersName = df.drop_duplicates()

In [137]:
outputFileName = "player_name_page_link_south_africa.csv"
df.to_csv(outputFileName)

### Lots of repeated names are there

In [138]:
df.nunique()

Player Name    10181
Page Link      10382
dtype: int64

## Make all the players name unique

In [2]:
outputFileName = "player_name_page_link_new_zealand.csv"

In [3]:
df = pd.read_csv(outputFileName)

In [4]:
uid = 0
prevPlayer = None

for i in range(len(df)):
    if prevPlayer == df['Player Name'][i]:
        uid += 1
    else:
        uid = 0
        prevPlayer = df['Player Name'][i]
        
    df.at[i,'Player Name'] = df['Player Name'][i] + "_" + str(uid)

In [5]:
outputFileNameNew = outputFileName.split('.')[0]+"_uniqueu_name.csv"
try:
    df = df.drop(['Unnamed: 0'], axis=1)
except:
    pass
df.to_csv(outputFileNameNew)

In [6]:
df.columns

Index(['Player Name', 'Page Link'], dtype='object')

In [7]:
df.nunique()

Player Name    3861
Page Link      3871
dtype: int64

# 2. Go to each of the pages and download data

In [ ]:
df = pd.read_csv('player_name_page_link.csv')

In [ ]:
try:
    os.mkdir("images")
except:
    pass

try:
    os.mkdir("details")
except:
    pass

In [ ]:
for i in range(len(df)):
    try:
        playerName = df.iloc[i]['Player Name']
        pageLink = df.iloc[i]['Page Link']

        # Go to the page
        driver.get(pageLink)

        # Sleep for 3 seconds
        time.sleep(3)

        try:
            imageDiv = driver.find_element(By.CSS_SELECTOR,'div.ds-ml-auto.ds-w-48.ds-h-48')
            imageLink = imageDiv.find_element(By.TAG_NAME,"img").get_attribute('src')

            # Download the image
            response = requests.get(imageLink)
            imageFileName = './images/'+playerName+'.png'
            jsonFileName = './details/'+playerName+'.json'
            with open(imageFileName, "wb") as f:
                f.write(response.content)

            # Save the JSON file
            text = driver.find_element(By.ID,"__NEXT_DATA__").get_attribute('innerHTML')
            fulljsonData = json.loads(text)
            json_object = json.dumps(fulljsonData, indent = 4) 
            data = fulljsonData['props']['appPageProps']['data']['player']

            # Serializing json
            json_object = json.dumps(data, indent=4)

            # Writing to sample.json
            with open(jsonFileName, "w") as outfile:
                outfile.write(json_object)
        except:
            # means no image is there so no need to peoceed further
            print(pageLink)
    except:
        print(i)